In [68]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords,wordnet
from nltk import pos_tag
import string
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.metrics import classification_report

In [31]:
df_train=pd.read_csv('twitter_x_y_train.csv')
df_train.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [52]:
df_test=pd.read_csv('twitter_x_test.csv')
df_test.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [48]:
df_train.describe(include='all')

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
count,1.098000e+04,10980,10980,31,10980,24,10980.000000,10980,776,10980,7430,7403
unique,NaN,3,6,3,6438,11,NaN,10851,632,10758,2658,78
top,NaN,negative,United,negative,JetBlueNews,Customer Service Issue,NaN,@united thanks,"[0.0, 0.0]",2015-02-24 11:43:05 -0800,"New York, NY",Eastern Time (US & Canada)
freq,NaN,6851,2928,24,43,9,NaN,6,131,3,125,2819
mean,5.692169e+17,NaN,NaN,NaN,NaN,NaN,0.080965,NaN,NaN,NaN,NaN,NaN
std,7.795438e+14,NaN,NaN,NaN,NaN,NaN,0.740303,NaN,NaN,NaN,NaN,NaN
min,5.675883e+17,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,5.685584e+17,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
50%,5.694753e+17,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
75%,5.698902e+17,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


In [49]:
df_test.describe(include='all')

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
count,3.660000e+03,3660,9,3660,8,3660.000000,3660,243,3660,2477,2417
unique,NaN,6,2,2805,6,NaN,3650,209,3635,1258,59
top,NaN,United,negative,JetBlueNews,Customer Service Issue,NaN,@AmericanAir spent $8 for the choppiest feed o...,"[0.0, 0.0]",2015-02-24 09:54:34 -0800,USA,Eastern Time (US & Canada)
freq,NaN,894,8,20,3,NaN,2,33,3,43,925
mean,5.692226e+17,NaN,NaN,NaN,NaN,0.087705,NaN,NaN,NaN,NaN,NaN
std,7.779030e+14,NaN,NaN,NaN,NaN,0.762048,NaN,NaN,NaN,NaN,NaN
min,5.675924e+17,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,5.685633e+17,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
50%,5.694842e+17,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
75%,5.698927e+17,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


In [35]:
df_train.isnull().sum()

tweet_id                      0
airline_sentiment             0
airline                       0
airline_sentiment_gold    10949
name                          0
negativereason_gold       10956
retweet_count                 0
text                          0
tweet_coord               10204
tweet_created                 0
tweet_location             3550
user_timezone              3577
dtype: int64

In [50]:
df_test.isnull().sum()

tweet_id                     0
airline                      0
airline_sentiment_gold    3651
name                         0
negativereason_gold       3652
retweet_count                0
text                         0
tweet_coord               3417
tweet_created                0
tweet_location            1183
user_timezone             1243
dtype: int64

In [53]:
train_tweets=df_train['text'].values

In [54]:
test_tweets=df_test['text'].values

In [55]:
print(train_tweets[0:5])

['@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled'
 '@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!'
 '@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS'
 "@SouthwestAir @dultch97 that's horse radish 😤🐴"
 '@united so our flight into ORD was delayed because of Air Force One, but the last flight to SBN is at 8:20, 5 mins from now we just landed.']


In [57]:
print(test_tweets[0:5])

["@AmericanAir In car gng to DFW. Pulled over 1hr ago - very icy roads. On-hold with AA since 1hr. Can't reach arpt for AA2450. Wat 2 do?"
 '@AmericanAir after all, the plane didn’t land in identical or worse) conditions at GRK according to METARs.'
 "@SouthwestAir can't believe how many paying customers you left high and dry with no reason for flight Cancelled Flightlations Monday out of BDL! Wow."
 '@USAirways I can legitimately say that I would have rather driven cross country than flown on US Airways.'
 '@AmericanAir still no response from AA. great job guys!']


In [58]:
sentiments=df_train['airline_sentiment'].values

In [59]:
print(sentiments[3:7])

['negative' 'negative' 'negative' 'negative']


In [20]:
stop=stopwords.words('english')
stop=set(stop)
stop.update(list(string.punctuation))
stop

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [21]:
lematizer=WordNetLemmatizer()


In [22]:
def get_pos(poso):
    if poso.startswith('J'):
        return wordnet.ADJ
    elif poso.startswith('N'):
        return wordnet.NOUN
    elif poso.startswith('V'):
        return wordnet.VERB
    elif poso.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean(words):
    cleaned=[]
    for w in words:
        if w.lower() not in stop:
            pos=pos_tag([w])
            clean_word=lematizer.lemmatize(w,get_pos(pos[0][1]))
            cleaned.append(clean_word.lower())
    return cleaned

In [61]:
train_tweets_splitted=[word_tokenize(tweet) for tweet in train_tweets]
train_cleaned_tweets_splitted=[clean(each_splitted_tweet) for each_splitted_tweet in train_tweets_splitted]
train_cleaned_tweets_splitted[0]

['southwestair',
 'schedule',
 'morning',
 '2',
 'day',
 'fact',
 'yes..not',
 'sure',
 'even',
 'flight',
 'one',
 'cancelled',
 'flightled']

In [62]:
test_tweets_splitted=[word_tokenize(tweet) for tweet in test_tweets]
test_cleaned_tweets_splitted=[clean(each_splitted_tweet) for each_splitted_tweet in test_tweets_splitted]
test_cleaned_tweets_splitted[0]

['americanair',
 'car',
 'gng',
 'dfw',
 'pulled',
 '1hr',
 'ago',
 'icy',
 'road',
 'on-hold',
 'aa',
 'since',
 '1hr',
 'ca',
 "n't",
 'reach',
 'arpt',
 'aa2450',
 'wat',
 '2']

In [63]:
train_joined_clean_tweets=[" ".join(e) for e in train_cleaned_tweets_splitted]
train_joined_clean_tweets[0]

'southwestair schedule morning 2 day fact yes..not sure even flight one cancelled flightled'

In [64]:
test_joined_clean_tweets=[" ".join(e) for e in test_cleaned_tweets_splitted]
test_joined_clean_tweets[0]

"americanair car gng dfw pulled 1hr ago icy road on-hold aa since 1hr ca n't reach arpt aa2450 wat 2"

In [119]:
count_vect=CountVectorizer(max_features=2000)
x_train=count_vect.fit_transform(train_joined_clean_tweets)
x_test=count_vect.transform(test_joined_clean_tweets)
y_train=sentiments

In [120]:
clf=RandomForestClassifier(n_estimators=1500, n_jobs=-1)
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)

In [121]:
np.savetxt(fname='predicted.csv', X=y_pred, fmt='%s')